# Класс анализа кредитных данных
**Цель**
<br>Цель этого класса - анализ кредитных данных с использованием методов машинного обучения. 

<br>Включает функции 
  - загрузки набора данных,
  - предварительной обработки категориальных признаков,
  - разделения данных,
  - масштабирования признаков,
  - обучения модели и оценки ее производительности.

<br>**Задание**
<br>Написать класс CreditDataAnalysis

<br>Методы
- Инициализация __init__ : Инициализирует класс необходимыми атрибутами.
- Загрузка и предобработка данных
  - load_credit_dataset : Загружает набор данных 'credit-g'
    <br>с использованием fetch_openml из scikit-learn.
  - encode_data(encoder) : Преобразует все категориальные признаки в числовые
    <br>значения с помощью указанного энкодера (LabelEncoder или OneHotEncoder).
    <br>Удаляет старые признаки из self.data и добавляет преобразованные.
- Подготовка данных
  - split_data(test_size) : Разделяет набор данных на обучающую и тестовую выборки.
- Масштабирование признаков
  - scale_features(scaler) : Масштабирует признаки набора данных с использованием
    <br>указанного метода масштабирования (по умолчанию: StandardScaler).
- Обучение модели и оценка
  - train_model(model) : Обучает модель машинного обучения на обучающих данных (по умолчанию: LogisticRegression).
  - evaluate_model(metric) : Оценивает обученную модель на тестовых данных с использованием указанной метрики.
- Сравнение эффекта масштабирования на производительность модели
  - compare_scaling_effect(scaler, model, encoder, metric) :
    <br>Сравнивает эффект масштабирования на производительность модели,
    <br>обучая модель до и после масштабирования.
    <br>Отображает точность до и после масштабирования, а также процент изменения эффекта масштабирования.

## Импорт библиотек

In [1]:
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# создание файла requirements.txt
# !pipreqsnb data_analysis_test.ipynb

## Проверка без использования класса

### Загрузка набора данных

In [3]:
data = fetch_openml(name='credit-g', parser="auto", version=2)
df = pd.DataFrame(data=data.data, columns=data.feature_names)
df['target'] = data.target

df.info()
df.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   checking_status         1000 non-null   category
 1   duration                1000 non-null   int64   
 2   credit_history          1000 non-null   category
 3   purpose                 1000 non-null   category
 4   credit_amount           1000 non-null   float64 
 5   savings_status          1000 non-null   category
 6   employment              1000 non-null   category
 7   installment_commitment  1000 non-null   int64   
 8   personal_status         1000 non-null   category
 9   other_parties           1000 non-null   category
 10  residence_since         1000 non-null   int64   
 11  property_magnitude      1000 non-null   category
 12  age                     1000 non-null   int64   
 13  other_payment_plans     1000 non-null   category
 14  housing                 1

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,target
482,<0,30,existing paid,furniture/equipment,3622.0,>=1000,>=7,4,female div/dep/mar,none,...,life insurance,57,none,rent,2,skilled,1,yes,yes,good
6,no checking,24,existing paid,furniture/equipment,2835.0,500<=X<1000,>=7,3,male single,none,...,life insurance,53,none,own,1,skilled,1,none,yes,good
36,no checking,48,critical/other existing credit,education,6110.0,<100,1<=X<4,1,male single,none,...,no known property,31,bank,for free,1,skilled,1,yes,yes,good


### Кодирование категориальных признаков

Выделение категориальных признаков

In [4]:
categorical_features = df.select_dtypes(include='category').columns

Кодирование при помощи LabelEncoder

In [5]:
label_encoder = LabelEncoder()

for feature in categorical_features:
    df[feature] = label_encoder.fit_transform(df[feature])

Кодирование при помощи OneHotEncoder

In [6]:
onehot_encoder = OneHotEncoder()

encoded_features = onehot_encoder.fit_transform(df[categorical_features])
feature_names = onehot_encoder.get_feature_names_out(categorical_features)
encoded_df = pd.DataFrame(encoded_features.toarray(), columns=feature_names)
df_encoded = pd.concat([df.drop(categorical_features, axis=1), encoded_df], axis=1)

### Разделение на выборки

In [7]:
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 20), (200, 20), (800,), (200,))

### Масштабирование данных

In [8]:
scaler = StandardScaler()
X_train_s = (
    pd.DataFrame(
        scaler.fit_transform(X_train), 
        columns=X_train.columns, 
        index=X_train.index)
)
X_test_s = (
    pd.DataFrame(
        scaler.transform(X_test), 
        columns=X_test.columns, 
        index=X_test.index)
)

### Обучение модели

Признаки без масштабирования

In [9]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

Масштабированные признаки

In [10]:
model_s = LogisticRegression(max_iter=1000)
model_s.fit(X_train_s, y_train)

LogisticRegression(max_iter=1000)

### Оценка качества модели

Признаки без масштабирования

In [11]:
y_pred = model.predict(X_test)
f"accuracy before scaling: {accuracy_score(y_test, y_pred)}"

'accuracy before scaling: 0.71'

Масштабированные признаки

In [12]:
y_pred_s = model_s.predict(X_test_s)
f"accuracy after scaling: {accuracy_score(y_test, y_pred_s)}"

'accuracy after scaling: 0.73'

### Выявление эффекта от масшатбирования признаков

In [13]:
print(f'Accuracy before scaling: {accuracy_score(y_test, y_pred)}')
print(f'Accuracy after scaling: {accuracy_score(y_test, y_pred_s)}')
print(f"""Scaling effect percentage: 
{((accuracy_score(y_test, y_pred_s) - accuracy_score(y_test, y_pred)) 
  / accuracy_score(y_test, y_pred)) * 100:.2f}%""")

Accuracy before scaling: 0.71
Accuracy after scaling: 0.73
Scaling effect percentage: 
2.82%


## Проведение проверки класса

In [14]:
from data_analysis import CreditDataAnalysis

credit_analysis = CreditDataAnalysis()
credit_analysis.compare_scaling_effect()

Accuracy before scaling: 0.71
Accuracy after scaling: 0.73
Scaling effect percentage: 2.82%


## Заключение
- написан класс CreditDataAnalysis
- произведена проверка работоспособности класса